# 국내도서 종합 베스트셀러

- http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1
- 책 제목, 저자, 출판사, 발행일, 정가, 할인가

In [1]:
# 관련 모듈 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

### 한페이지의 책 정보

In [18]:
# 1) 페이지 주소 요청 => 텍스트
url = 'http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1'
response = requests.get(url)
html_str = response.text
#html_str

In [19]:
# 2) BeautifulSoup 객체화
soup = BeautifulSoup(html_str, 'html.parser') # parser 보기좋게(유의미하게?) 정리
#soup

In [20]:
# 3) 필터링(반복되는 요소의 선택자)
# #category_layout tbody tr
tr_list = soup.select('#category_layout tr:nth-child(odd)')
len(tr_list)

20

In [10]:
tr_list[0]

<tr>
<td class="num" id="goods1">1.</td>
<td class="image" rowspan="2">
<div class="goodsImgW">
<a href="/Product/Goods/102789938">
<img alt="달러구트 꿈 백화점 2" src="http://image.yes24.com/goods/102789938/S">
</img></a>
<a href="javascript:yes24GU.openPreviewCheck(102789938);"><img alt="미리보기" src="http://image.yes24.com/sysimage/categoryN/b_preview.gif"/></a>
</div>
</td>
<td class="goodsTxtInfo">
<p>[도서]   <a href="/Product/Goods/102789938">달러구트 꿈 백화점 2</a> : 단골손님을 찾습니다 
            
            <a href="/Product/Goods/102789938" target="_blank"><img alt="새창" src="http://image.yes24.com/sysimage/bestseller/b_newwin.gif"/></a></p>
<div class="aupu">
<a href="http://www.yes24.com/SearchCorner/Result?domain=ALL&amp;author_yn=Y&amp;query=&amp;auth_no=305800" target="_blank">이미예</a> 저  
                
                
                    | <a href="http://www.yes24.com/SearchCorner/Result?domain=ALL&amp;company_yn=Y&amp;query=%c6%d1%c5%e4%b8%ae%b3%aa%c0%ce">팩토리나인</a>
                
        

In [21]:
print(tr_list[0].select_one('p a').text) # 제목
base_url = 'https://yes24.com'
print(base_url + tr_list[0].select_one('p a')['href']) # url
print(tr_list[0].select_one('.aupu').text.split('|')) # 작가, 출판사, 출판일
# 각 요소에 대해서 공백과 개행문자 삭제
temp = list(map(lambda x:x.strip(),tr_list[0].select_one('.aupu').text.split('|')))
print(temp)
print(temp[0]) # 작가
print(temp[1]) # 출판사
print(temp[2]) # 출판일

달러구트 꿈 백화점 2
https://yes24.com/Product/Goods/102789938
['\n이미예 저  \r\n                \r\n                \r\n                    ', ' 팩토리나인\r\n                \r\n                    ', ' 2021년 07월\r\n                 \r\n            ']
['이미예 저', '팩토리나인', '2021년 07월']
이미예 저
팩토리나인
2021년 07월


In [26]:
# td.goodsTxtInfo > p:nth-child(3)
print(tr_list[0].select_one('.goodsTxtInfo > p:nth-child(3)').text)
print(tr_list[0].select_one('.goodsTxtInfo > p:nth-child(3)').text.split()[0])
#print(tr_list[0].select_one('.goodsTxtInfo > p:nth-child(3)').text.split()[1])

# .goodsTxtInfo > p:nth-child(3) > span.priceB
print(tr_list[0].select_one('.goodsTxtInfo > p:nth-child(3) > span.priceB').text)

13,800원 →12,420원(10% 할인) | YES포인트 690원(5% 지급)
13,800원
12,420원


In [27]:
# 4) 2차원의 책 정보로 저장하기
book_list1 = []
base_url = 'http://yes24.com'

for tr in tr_list:
    title = tr.select_one('p a').text
    url = base_url + tr.select_one('p a')['href']
    temp = list(map(lambda x:x.strip(), tr.select_one('.aupu').text.split('|')))
    writer = temp[0]
    publisher = temp[1]
    published_date = temp[2]
    price = tr.select_one('.goodsTxtInfo > p:nth-child(3)').text.split()[0]
    discount_price = tr.select_one('.goodsTxtInfo > p:nth-child(3) > span.priceB').text
    book_list1.append([title, writer, publisher, published_date, price, discount_price, url])

len(book_list1)

20

In [28]:
df1 = pd.DataFrame(book_list1, 
                   columns=['title', 'writer', 'publisher', 'published_date', 'price', 
                            'discount_price', 'url'])
df1.head()


,title,writer,publisher,published_date,price,discount_price,url
0,달러구트 꿈 백화점 2,이미예 저,팩토리나인,2021년 07월,"13,800원","12,420원",http://yes24.com/Product/Goods/102789938
1,달러구트 꿈 백화점,이미예 저,팩토리나인,2020년 07월,"13,800원","12,420원",http://yes24.com/Product/Goods/91065309
2,작별하지 않는다,한강 저,문학동네,2021년 09월,"14,000원","12,600원",http://yes24.com/Product/Goods/103495056
3,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키 저/권영주 역,모모,2021년 06월,"14,000원","12,600원",http://yes24.com/Product/Goods/102360203
4,인생은 실전이다,"신영준,주언규(신사임당) 저",상상스퀘어,2021년 09월,"18,500원","16,650원",http://yes24.com/Product/Goods/103598738


# 퀴즈1
- book_print(시작페이지, 마지막페이지) 함수를 호출하면
- 데이터프레임 형태로 출력되도록 한다.
```
# 함수 호출 시
book_print(1,5)
# 데이터 프레임 형태로 1~5 페이지 전체의 책 정보가 출력
```

# 퀴즈2
- 전체 페이지의 책 정보가 데이터프레임으로 저장되어 출력되도록 프로그래밍하여라.

In [31]:
# 퀴즈1

def book_print(start,last):
    book_listQ = []
    base_url = 'http://www.yes24.com'
    for i in range(start,last+1):
        url = f'http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber={i}'
        response = requests.get(url)
        html_str = response.text
    
        soup = BeautifulSoup(html_str, 'html.parser') # parser 보기좋게(유의미하게?) 정리

        tr_list = soup.select('#category_layout tr:nth-child(odd)')
    
        for tr in tr_list:
            title = tr.select_one('p a').text
            url = base_url + tr.select_one('p a')['href']
            temp = list(map(lambda x:x.strip(), tr.select_one('.aupu').text.split('|')))
            writer = temp[0]
            publisher = temp[1]
            published_date = temp[2]
            price = tr.select_one('.goodsTxtInfo > p:nth-child(3)').text.split()[0]
            discount_price = tr.select_one('.goodsTxtInfo > p:nth-child(3) > span.priceB').text
            book_listQ.append([title, writer, publisher, published_date, price, discount_price, url])

    dfQ = pd.DataFrame(book_listQ, 
                           columns=['title', 'writer', 'publisher', 'published_date', 'price', 
                                    'discount_price', 'url'])
    return dfQ


In [32]:
book_print(1,5)

,title,writer,publisher,published_date,price,discount_price,url
0,달러구트 꿈 백화점 2,이미예 저,팩토리나인,2021년 07월,"13,800원","12,420원",http://www.yes24.com/Product/Goods/102789938
1,달러구트 꿈 백화점,이미예 저,팩토리나인,2020년 07월,"13,800원","12,420원",http://www.yes24.com/Product/Goods/91065309
2,작별하지 않는다,한강 저,문학동네,2021년 09월,"14,000원","12,600원",http://www.yes24.com/Product/Goods/103495056
3,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키 저/권영주 역,모모,2021년 06월,"14,000원","12,600원",http://www.yes24.com/Product/Goods/102360203
4,인생은 실전이다,"신영준,주언규(신사임당) 저",상상스퀘어,2021년 09월,"18,500원","16,650원",http://www.yes24.com/Product/Goods/103598738
...,...,...,...,...,...,...,...
95,코스모스,칼 세이건 저/홍승수 역,사이언스북스,2006년 12월,"18,500원","16,650원",http://www.yes24.com/Product/Goods/2312211
96,모든 날에 모든 순간에 위로를 보낸다,글배우 저,강한별,2021년 04월,"13,800원","12,420원",http://www.yes24.com/Product/Goods/99129934
97,흔한남매 과학 탐험대 1 우주,"흔한남매 원저/이재국,이현진 글/김덕영 그림/정현철,권경아,최진수,흔한컴퍼니 감수/...",주니어김영사,2021년 05월,"12,800원","11,520원",http://www.yes24.com/Product/Goods/101507429
98,설민석의 삼국지 대모험 3,단꿈아이 글/스튜디오 담 그림,단꿈아이,2021년 08월,"11,000원","9,900원",http://www.yes24.com/Product/Goods/103346349


In [33]:
# 퀴즈2

book_listQ2 = []
base_url = 'http://www.yes24.com'
page = 0

while True:
    page += 1
    
    url = url = f'http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber={page}'
    response = requests.get(url)
    html_str = response.text
    
    soup = BeautifulSoup(html_str, 'html.parser') # parser 보기좋게(유의미하게?) 정리

    tr_list = soup.select('#category_layout tr:nth-child(odd)')
    
    if tr_list:
        for tr in tr_list:
            rank = tr.find(class_='num').text.split('.')[0]
            title = tr.select_one('.goodsTxtInfo p a').text
            temp = list(map(lambda x:x.strip(), tr.find(class_='aupu').text.strip().split('|') ))
            writer = temp[0]
            publisher = temp[1]
            published_date = temp[2]
            price = tr.select_one('p:nth-child(3)').text.split()[0].strip()
            discount_price = tr.select_one('p:nth-child(3)').text.strip().split('→')[1].split('원')[0]+'원'
            book_listQ2.append([rank, title, writer, publisher, published_date, price, discount_price])
    else:
        break
    
    
field_list = ['rank', 'title', 'writer', 'publisher', 'published_date', 'price', 'discount_price']
dfQ2 = pd.DataFrame(book_listQ2, columns = field_list)
dfQ2.head()    
    
    
    
    

,rank,title,writer,publisher,published_date,price,discount_price
0,1,달러구트 꿈 백화점 2,이미예 저,팩토리나인,2021년 07월,"13,800원","12,420원"
1,2,달러구트 꿈 백화점,이미예 저,팩토리나인,2020년 07월,"13,800원","12,420원"
2,3,작별하지 않는다,한강 저,문학동네,2021년 09월,"14,000원","12,600원"
3,4,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키 저/권영주 역,모모,2021년 06월,"14,000원","12,600원"
4,5,인생은 실전이다,"신영준,주언규(신사임당) 저",상상스퀘어,2021년 09월,"18,500원","16,650원"
